In [96]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [97]:
def plot_signs(images, labels=None, rows=2, cols=4, rnd=True):

    nimg = rows*cols
    if rnd:
        selection = np.random.choice(images.shape[0], nimg, replace=False)
    else:
        selection = range(nimg)
        
    images_to_show = images[selection]
    
    f, axarr = plt.subplots(rows, cols, figsize=(12, 3*rows))
    for row in range(rows):
        for col in range(cols):
            index = (row*cols)+col
            ax = axarr[row,col]  if rows > 1 else axarr[col] 
            ax.imshow(images_to_show[index])
            ax.axis('off')
            
            if labels is not None:
                label_id = labels[selection][index]
                ax.set_title('%s - %s' % (label_id, signnames[label_id]), fontsize=8)

In [98]:
def normalize(images, newMax=1., newMin=0., oldMax=255, oldMin=0):
    images = images.astype('float32')
    return (images - oldMin) * ((newMax - newMin)/(oldMax - oldMin)) + newMin

In [99]:
training_file = 'data/train.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_train = normalize(X_train)

In [100]:
rotation = 20
trans_range = 0.2
channel_shift = 0.05
datagen = ImageDataGenerator(rotation_range=rotation,                           
                             channel_shift_range=channel_shift,
                             shear_range=trans_range,
                             zoom_range=trans_range,
                             width_shift_range=trans_range,
                             height_shift_range=trans_range,
                             fill_mode='nearest')

datagen.fit(X_train)

In [101]:
batch_size = X_train.shape[0]
nb_transoformations = 4

for i, (X_batch, y_batch) in enumerate(datagen.flow(X_train, y_train, batch_size=batch_size)):       
    if i == nb_transoformations:
        break
    
    with open('data/train_jittered_%s.p' % i, 'ab') as f:
        pickle.dump({'features':X_batch, 'labels':y_batch}, f)